# Implementation of k-NN

This notebook was written by Gael Lederrey and Tim Hillel (tim.hillel@epfl.ch) for the Decision-aid methodologies in transportation course at EPFL (http://edu.epfl.ch/coursebook/en/decision-aid-methodologies-in-transportation-CIVIL-557).

Please contact before distributing or reusing the material below.

## Overview

In this notebook, you will learn how to implement the $k$-NN algorithm. This notebook is separated in two parts:

1. Implementing the binary case $k$-NN classifier
2. Implementing the multinomial case.

### Setup

We start by loading the dataset and the different libraries that are required for the exercises. We randomly sample 10000 elements from the original dataset, to make computation faster.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_full = pd.read_csv('data/dataset.csv')

# We sample the dataset to decrease computation time
df = df_full.sample(10000, random_state = 123)

## Implementation of k-NN for binary case

### Data preparation

We start this exercise by selecting the features and output. In this case, we want a binary label which represents if a trip is made by driving (*1*) or not (*0*).

The label is currently a string, so we will map it to numeric data.

We will also select only four **features** from the data (again, in order to reduce computation time).

In [3]:
# Dictionary used to transform the string in 
# the travel_mode for the binary output
str_to_val = {
    'drive': 1,
    'pt': 0,
    'cycle': 0,
    'walk': 0
}

# Output
y = df['travel_mode'].replace(str_to_val).values

# Features (4 were selected)
x = df[['age', 'car_ownership', 'distance', 'female']].values

### Train-test split

In [4]:
# We split the output and features into a train and a test set by an (approximate) ratio of 0.8
np.random.seed(123)
msk = np.random.rand(len(df)) < 0.8

x_train = x[msk]
x_test = x[~msk]

y_train = y[msk]
y_test = y[~msk]

### Distance

We need to compute the distance between two points in the space.

*Note* You can either handle 2D arrays in your function, or handle this case with a for loop.

We can either calculate the solution using first principles:

In [5]:
# First principles solution:
def euclidean_distance(array1, array2):
    # Enter your code below
    array1=np.array(array1)
    array2=np.array(array2)
    if len(array2.shape)==1:
        dist = np.sqrt(np.sum([(i-j)**2 for i, j in zip(array1, array2)])) 
        #zip combines two arrays in one loop
    else:
        dist = [np.sqrt(np.sum([(i-j)**2 for i, j in zip(array1, arr)])) for arr in array2]
    return dist

...or we could use instead the _norm_ function from numpy to calculate the Euclidean distance:

In [6]:
def euclidean_distance_numpy(array1, array2):
    # Enter your code below
    array1 = np.array(array1)
    array2 = np.array(array2)
    ax = 1
    if len(array1.shape) == 1 and len(array2.shape) == 1:
        ax = None
    return np.linalg.norm(array1-array2, axis=ax)

**Verify your answer here:**

`euclidean_distance([3, -2, -5, 9], [-1, 4, 2, 0])) = 13.491`

In [7]:
arr1 = [3, -2, -5, 9]
arr2 = [-1, 4, 2, 0]

euclidean_distance(arr1, arr2)

13.490737563232042

In [8]:
np.sqrt((3-10)**2+(-2-7)**2+(-5-6)**2+(9--7)**2)

22.516660498395403

    euclidean_distance([3, -2, -5, 9], 
                       [[-1, 4, 2, 0],[10,7,6,-7]]) = array([13.491, 22.517])

In [9]:
euclidean_distance([3, -2, -5, 9], 
                   [[-1, 4, 2, 0], [10, 7, 6, -7]])

[13.490737563232042, 22.516660498395403]

### Find the k nearest neighbours

For this function, we need to find the `k` nearest neighbours out of the `known_points` for a single `candidate`. The `candidate` is an array (with $n$ features), and it needs to be compared to the rest of the `known_points` (in the training set). 

The function should return the indices of the `k` closest neighbours.

In [10]:
def get_k_nearest_neighbours_fast(candidate, known_points, k):
    # Enter your code below

    # Get the distances
    distances = euclidean_distance_numpy(known_points, candidate)
    
    # Sort the distances
    neighbours_idx = distances.argsort()
    
    return neighbours_idx[:k]

**OR**

In [11]:
def get_k_nearest_neighbours_slow(candidate, known_points, k):
    # Enter your code below

    # Get the distances
    # distances = []
    # for p in known_points:
    #     distances.append(euclidean_distance_1D(p, candidate))
        
    distances = np.array([euclidean_distance(p, candidate) for p in known_points])
    
    # Sort the distances
    neighbours_idx = distances.argsort()
    
    return neighbours_idx[:k]

**Verify your answer here:**

`get_k_nearest_neighbours(x_test[0], x_train, 5) = array([2648, 3071, 7607, 7686, 3466])`

In [12]:
neighbours = get_k_nearest_neighbours_slow(x_test[0], x_train, 5)
neighbours

array([2648, 3071, 7607, 7686, 3466])

### Find the class

Now that we know the closest neighbours, we need to compute the class (output) for a given instance. To do this, we need to count the number of times the class of a neighbour appears. Then, we can select the class with the highest count.

In [13]:
def get_class(neighbours, y_true):
    # Enter your code below
    # Binomial
    return int(np.mean(y_true[neighbours]) > 0.5)
    # Any class labels
    

**Verify your answer here:**

`get_class(neighbours, y_train) = 1`

In [14]:
get_class(neighbours, y_train)

1

You can verify that three neighbours have the class `1` and two have the class `0`.

In [15]:
y_train[neighbours]

array([1, 1, 0, 1, 0])

### Compute the accuracy

We need to implement the computation of the accuracy. Recall that the accuracy is the proportion of the predictions we get right (i.e. where our **prediction** is equal to the **ground truth**) 

In [16]:
def compute_accuracy(y_true, y_pred):
    # Enter your code below
    return np.mean(np.array(y_true)==np.array(y_pred))*100

**Verify your answer here:**

`compute_accuracy([0, 1, 1, 0, 1], [0, 1, 1, 1, 0]) = 60.0`

In [17]:
compute_accuracy([0, 1, 1, 0, 1], [0, 1, 1, 1, 0])

60.0

## Test your implementation of k-NN

We can now test our implementation of k-NN. To do so, we need to combine the functions we developed the following order: 

```
For each observation in the test set:
    Get the k nearest neighbours
    Get the class of the current observation
    Add the prediction to a vector
    
Compute the accuracy
    
```

With `k=5`, you should get an accuracy of `60.399%`.


In [18]:
%%time 
# ^- Show the time

# Enter your code below
y_pred = []
k = 5
for x in x_test:
    neigh = get_k_nearest_neighbours_slow(x, x_train, k)
    class_ = get_class(neigh, y_train)
    y_pred.append(class_)

accuracy = compute_accuracy(y_test, y_pred)
print('Accuracy: {:.3f}%'.format(accuracy))

Accuracy: 60.399%
CPU times: user 7min 41s, sys: 4.89 s, total: 7min 46s
Wall time: 7min 49s


## k-NN for multinomial cases

First, we need to change the data to add a multinomial output.

Indeed, we would like to see if we can classify between all four transport modes.

In [19]:
# Dictionnary used to transform the string in 
# the travel_mode to an integer
str_to_val = {
    'walk': 0,
    'cycle': 1,
    'pt': 2,
    'drive': 3
}

# Output
y = df['travel_mode'].replace(str_to_val).values

In [20]:
y_train_mult = y[msk]
y_test_mult = y[~msk]

### Modification of the code

Modify any functions that you implemented earlier to take into account the multinomial outputs.

In [23]:
# Need to make get_class work for the mutinomial case
def get_class(neighbours, y_true):
    values, counts = np.unique(y_true[neighbours],return_counts=True)
    i_max=np.argmax(counts)
    return values[i_max]

###### **Verify your answer here:**

With `k=5`, you should now get an accuracy of `53.756%`.

In [22]:
%%time
# ^- Show the time

# Enter your code below
y_pred = []
k = 5
for x in x_test:
    neigh = get_k_nearest_neighbours_fast(x, x_train, k)
    class_ = get_class(neigh, y_train_mult)
    y_pred.append(class_)

accuracy = compute_accuracy(y_test_mult, y_pred)
print('Accuracy: {:.3f}%'.format(accuracy))

Accuracy: 53.756%
CPU times: user 2.48 s, sys: 15.6 ms, total: 2.5 s
Wall time: 2.46 s


As we can see, the fast implementation took only 2.5 seconds while the slow implementation took nearly 8 minutes. This is due to the `for` loops (list compre in the slow implementation. Indeed, Python's `for` loops are especially slow and numpy is well optimized. (Foor for thought: https://dev.to/duomly/loops-in-python-comparison-and-performance-4f2m) So, we advise you to always use a library if you can.